In [1]:
import warnings
from importlib import reload

warnings.filterwarnings('ignore')

In [2]:
import openai
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown


FAMAGA_DOWNLOAD_HTML_URL = "https://test-api.famaga.org/imap"
FAMAGA_DOWNLOAD_HTML_TOKEN = "YXBpZmFtYWdhcnU6RHpJVFd1Lk1COUV4LjNmdERsZ01YYlcvb0VFcW9NLw"
EXTRACTED_DEALS_DATABASE_PATH = 'extracted_deals_messaging.db'
CLIENTS_HISTORY_PATH = r'C:\Users\MGroup\Documents\products.json'
GPT_DB_LOGGER_PATH = 'sqlite:///prompt_versions.db'

AGENTS_API_URL = 'http://localhost:8000'

In [24]:
from importlib import reload
import clients

reload(clients)
# reload(clients.msg_html_extracter)
# reload(clients.gpt_database_logger)
# reload(clients.clients_deals_history)

from clients.agents import AgentsAPIClient
from clients.client_statistics import ClientStatisticsService
from clients.clients_deals_history import ClientDealsHistoryRepository
from clients.email_downloader import EmailDownloader
from clients.extracted_deals import ExtractedDealsRepository
from clients.gpt_database_logger import GPTDatabaseLogger
from clients.msg_html_extracter import MsgHtmlExtracter

print('Updated')

Updated


In [25]:
deals_rep = ExtractedDealsRepository(EXTRACTED_DEALS_DATABASE_PATH)
client_deals_rep = ClientDealsHistoryRepository(CLIENTS_HISTORY_PATH)
client_statistics = ClientStatisticsService()
downloader = EmailDownloader(FAMAGA_DOWNLOAD_HTML_URL, FAMAGA_DOWNLOAD_HTML_TOKEN)
client = AgentsAPIClient(AGENTS_API_URL)
db_logger = GPTDatabaseLogger(GPT_DB_LOGGER_PATH)

In [56]:
import json
import re

def select_json_block(text: str):
    match = re.search(r"```json\n([\s\S]*?)\n```", text)
    if match:
        json_data = match.group(1)
    else:
        raise ValueError("No valid JSON data found in the string.")

    return json.loads(json_data)

In [70]:
from typing import List, Literal, Optional
from pydantic import BaseModel, Field
import json


class Intent(BaseModel):
    intent: str
    sub_intent: str


class Message(BaseModel):
    id: str  # Adjust the type as needed (e.g., int for integer IDs)
    body: str
    sign: Optional[str] = None
    from_: Literal['customer', 'manager'] = Field(..., alias='from')
    intents: List[Intent]
    metadata_keys: List[str]

    class Config:
        allow_population_by_field_name = True

In [5]:
msg = """
Dear Sir or Madam,
Please advise if stators and rotors are available for the attached. The pump appears to be very similar to a Mono pump
Kind regards
Austen Brunton BEng (Hons), MIMarEST
Technical Fleet Manager
WYM Group Ltd
Incorporating Wilson Yacht Management and Wilson Ship Management
New Email Address: austen@wymgroup.com
Web: www.wymgroup.com
Phone: +44 (0) 1482 648322
Mob: +44 (0) 7760665835
"""

### Get message intents

In [77]:
class AgentExecutor:
    def __init__(self, gpt_logger_path: str):
       self.db_logger = GPTDatabaseLogger(gpt_logger_path)

    def get_message_intents(self, msg: str):
        response = self.db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
        { "role": "user", "content": f"""
Please attach to each message in chat history tags, annotations, indexes, intents to better classification and search.
The message could have one or more intents.

Message:
```
Dear Kristine! Could you give us discount?
```

You can choose one of the following intents or figure out your own.
   - Order Processing -> Order Placement Confirmation
   - Order Processing -> Order Acceptance Confirmation
   - Pricing and Quotes -> Discount Inquiry
   - Pricing and Quotes -> Transport Cost Inquiry
   - Delivery and Shipping -> Delivery Method Discussion
   - Delivery and Shipping -> Discussing Logistics
   - Product Inquiry -> Availability Check

Also please define which metadata entites should be find from message.
The available keys:
- discount_inquiry: the metadata of discount discussions, could be specified at messages with Pricing and Quotes -> Discount Inquiry intent
- client_info: info about client email, phone, office, country, address
- part_inquiry: the info about parts and components (part number, brand, model, qty)


Put the result at ```json``` format, like:
```json
    {{
        "id": "<message_id>",
        "body": "<message body>", //only body without sign
        "sign": "<signature>", //message signature
        "from": "<message from>", //this should be one of two values: customer or manager
        "intents": [
            {{
                "intent": "<main intent>",
                "sub_intent": "<sub intent>"
            }}
        ],
        "metadata_keys": [ 'key1', 'key2']
    
    }}
```
Please put new lines symbols if it suitable for context \\n at body and sign fields.
    """}                             
    ])
        response_raw_json = select_json_block(response)
        parsed_messages = Message(**response_raw_json)
        return parsed_messages

In [217]:
msg3 = """
Hello Ali,

Hope this email finds you well.

Please be informed that we had submitted our offer to the end user and got feedback from them that our prices were on the higher side by 45% from the prices other vendors quoted to them.

Hence we request you to please recheck the prices and help us so we will be able to win this bid.
"""

In [218]:
get_message_intents(msg3)

```json
[
    {
        "id": "1",
        "body": "Hello Ali,\n\nHope this email finds you well.\n\nPlease be informed that we had submitted our offer to the end user and got feedback from them that our prices were on the higher side by 45% from the prices other vendors quoted to them.\n\nHence we request you to please recheck the prices and help us so we will be able to win this bid.",
        "sign": "",
        "from": "customer",
        "intents": [
            {
                "intent": "Pricing and Quotes",
                "sub_intent": "Discount Inquiry",
                "keywords": ["offer", "prices", "higher", "vendors", "quoted", "recheck", "win", "bid"]
            }
        ]
    
    }
]
```

--------------------

Note saved without feedback. ID: aeb8241d-2497-4599-9a07-b17eb332049a
Input tokens: 336 Output tokens: 174 Total price: 0.02$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

### Get message details

In [181]:
def get_details_from_message(msg):
    response = db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
    { "role": "user", "content": f"""
Try to extract from text brand name, amount, detail name, part number from the text. Also recognize country by text.
<<<>>>
Dear Sir or Madam,\nPlease advise if stators and rotors are available for the attached. 
The pump appears to be very similar to a Mono pump\nKind regards
<<<>>>

If you cannot recognize specified parameters please put `null` value.

---

Your response should be a list of comma separated values, eg: `foo, bar, baz`

The output should be a markdown code snippet formatted in the following adr, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{{
    "parts": [
        {{
           "amount": int // This is the amount of details
           "brand_name": string // This is thr brand  name of detail
           "part_number": string // This is the part number of detail
        }}
    ],
    "client": {{
        "country": string // This is the country of detail,
        "domain": string // customer company domain
        "email": string // customer email,
        "office_country": string // country of customer office
    }}
}}
```
    """}                             
    ])
    return response

<>:35: SyntaxWarning: invalid escape sequence '\`'
<>:35: SyntaxWarning: invalid escape sequence '\`'
C:\Users\MGroup\AppData\Local\Temp\ipykernel_28008\2480512955.py:35: SyntaxWarning: invalid escape sequence '\`'
  """}


In [182]:
parsed_messages = get_message_intents(msg)

In [ ]:
msg_details = get_details_from_message(msg)

### Build SQL query

In [122]:
def build_sql_query(msg_details, intents):
    response = db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
    { "role": "user", "content": f"""
Here is the table.

TABLE product_inquiries (
    intent TEXT,
    sub_intent TEXT,
    is_brand_name BOOLEAN,
    is_part_number BOOLEAN,
    action TEXT
)

Please build sql query that select one or more rows by presented data. Please ignore action field at sql request condition. Database: sqlite

{intents}

Message details:
{msg_details}
---

Put the result at ```json``` format, like:
```sql
SELECT * FROM table_name
WHERE field1 = 'value_2'
```
    """}                             
    ])
    match = re.search(r"```sql\n([\s\S]*?)\n```", response)
    if match:
        return match.group(1)
    else:
        raise ValueError("No valid JSON data found in the string.")

    return json.loads(json_data)
    return response_raw_json

In [ ]:
intents_str = '\n'.join([f'{intent.intent} -> {intent.sub_intent}' for intent in parsed_messages[0].intents])
intents_str
raw_sql = build_sql_query(msg_details, intents_str)

In [45]:
for row in rows:
    action = row[4]
    print(action)

ask_specify_part_number


### Ask specify part number

In [52]:
response = db_logger.create_completion(temperature=0.3, tags='intent_classifier', model='gpt-4', messages=[
{ "role": "user", "content": f"""
You are sales manager that sale parts and components for manufacturers.
As a Senior sales manager, please ask customer to specify part number for part. When you build your own message please read the examples of 
other sales managers [EXAMPLE MESSAGES].

Metadata:
{{
    "parts": [
        {{
           "amount": null,
           "brand_name": "Mono",
           "part_number": null
        }}
    ],
    "client": {{
        "country": null,
        "domain": null,
        "email": null,
        "office_country": null
    }}
}}

The example of previous messages from sales use it to build your own:

[EXAMPLE MESSAGES]
Message 1: ```
Dear Sir / Madam,
Thank you for your request!
Could you please tell us the article number?
```
[/EXAMPLE MESSAGES]
"""}                             
])

Dear Valued Customer,

Thank you for your interest in our Mono brand parts. In order to assist you more efficiently, could you please provide us with the specific part number you are interested in?

Looking forward to your response.

--------------------

Note saved without feedback. ID: 13bafa8b-f97e-481b-b50e-6ed5a9d37a4c
Input tokens: 169 Output tokens: 45 Total price: 0.01$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

### Google sheet reader

In [6]:
import sqlite3
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pydantic import BaseModel, Field, field_validator


class ProductInquiry(BaseModel):
    intent: str
    sub_intent: str
    is_brand_name: bool
    is_part_number: bool
    action: str

    @field_validator('is_brand_name', 'is_part_number')
    def convert_int_to_bool(cls, v):
        if isinstance(v, bool):
            return v  # In case the input is already a boolean
        if v in (1, 'TRUE', 'True', 'true'):
            return True
        elif v in (0, 'FALSE', 'False', 'false'):
            return False
        raise ValueError('Must be a boolean or equivalent integer (0 or 1)')



class GoogleSheetToSQLiteConverter:
    def __init__(self, spreadsheet: str, credentials_path: str):
        scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)
        self.spreadsheet = spreadsheet
        self.client = gspread.authorize(creds)
        
    def get_sheet_data(self, sheet: str):
        worksheet = self.client.open(self.spreadsheet).worksheet(sheet)
        records = worksheet.get_all_records()

        return records

### Init sql db

In [279]:
import sqlite3


def init_sqlite_db(conn, product_inquiries, discount_branch):
    cursor = conn.cursor()
    
    product_inquiries_data = [
        [
            inquiry.intent,
            inquiry.sub_intent,
            int(inquiry.is_brand_name),  # Convert boolean to int
            int(inquiry.is_part_number),  # Convert boolean to int
            inquiry.action
        ]
        for inquiry in product_inquiries
    ]

    discount_data = [
        [
            discount_branch.sub_intent,
            discount_branch.is_margin_sufficient,
            discount_branch.is_discount_specified,
            discount_branch.is_customer_bought_previously,
            discount_branch.action
        ]
    ]
    
    print()
    cursor.execute('''CREATE TABLE product_inquiries (
        intent TEXT,
        sub_intent TEXT,
        is_brand_name BOOLEAN,
        is_part_number BOOLEAN,
        action TEXT
    )''')
    
    cursor.executemany('INSERT INTO product_inquiries VALUES (?, ?, ?, ?, ?)', product_inquiries_data)
    conn.commit()
    print('Data was added to db successfully.')

## Parse discount sheet fields

In [7]:
def map_googl_sheet_bool_to_int(v):
    if isinstance(v, bool):
        return v  # In case the input is already a boolean
    if v in (1, 'TRUE', 'True', 'true'):
        return True
    elif v in (0, 'FALSE', 'False', 'false'):
        return False
    raise ValueError('Must be a boolean or equivalent integer (0 or 1)')

In [8]:
class Discount(BaseModel):
    sub_intent: str
    is_margin_sufficient: bool
    is_discount_specified: bool
    is_customer_bought_previously: bool
    action: str

    @field_validator('is_margin_sufficient', 'is_discount_specified', 'is_customer_bought_previously')
    def convert_int_to_bool(cls, v):
        return map_googl_sheet_bool_to_int(v)

In [272]:
ghconv = GoogleSheetToSQLiteConverter(
    spreadsheet='Famaga Knowledge Map', 
    credentials_path=r'C:\Users\MGroup\Downloads\langchain-400510-ac38916fe0c8.json')

sheet_data = ghconv.get_sheet_data('discount')


In [15]:
from typing import List


class FamagaKBRepo:
    def __init__(self, spreadsheet, credentials_path):
        self.ghconv = GoogleSheetToSQLiteConverter(
            spreadsheet=spreadsheet, 
            credentials_path=credentials_path)

    def get_discount_branch(self) -> List[Discount]:
        sheet_data = self.ghconv.get_sheet_data('discount')
        discount_branch = [
            Discount(
                sub_intent = item['Sub-intent'],
                is_margin_sufficient = item['Is margin still above 10%?'],
                is_discount_specified = item['Specified dicount, at %'],
                is_customer_bought_previously = item['Did customer buy the same part previously?'],
                action = item['Action']
            )
            for item in sheet_data
        ]
        return discount_branch
        


In [19]:
import sqlite3


class MemoryKB:
    def __init__(self):
        self.conn = sqlite3.connect(':memory:')
        self.cursor = self.conn.cursor()

    def create_discount_table(self, discounts):
        table_name = 'discount'
        discount_data = [
            [
                discount.sub_intent,
                discount.is_margin_sufficient,
                discount.is_discount_specified,
                discount.is_customer_bought_previously,
                discount.action
            ]
            for discount in discounts
        ]

        self.cursor.execute(f'''CREATE TABLE {table_name} (
            sub_intent TEXT,
            is_margin_sufficient BOOLEAN,
            is_discount_specified BOOLEAN,
            is_customer_bought_previously BOOLEAN,
            action TEXT
        )''')
        
        self.cursor.executemany(f'INSERT INTO {table_name} VALUES (?, ?, ?, ?, ?)', discount_data)
        self.conn.commit()
        print('Data was added to db successfully.')

    def create_product_inquiries_table(self, product_inquiries):
        table_name = 'product_inquiries'
        product_inquiries_data = [
            [
                inquiry.intent,
                inquiry.sub_intent,
                int(inquiry.is_brand_name),  # Convert boolean to int
                int(inquiry.is_part_number),  # Convert boolean to int
                inquiry.action
            ]
            for inquiry in product_inquiries
        ]
    
        self.cursor.execute(f'''CREATE TABLE {table_name} (
            intent TEXT,
            sub_intent TEXT,
            is_brand_name BOOLEAN,
            is_part_number BOOLEAN,
            action TEXT
        )''')
        
        self.cursor.executemany(f'INSERT INTO {table_name} VALUES (?, ?, ?, ?, ?)', product_inquiries_data)
        self.conn.commit()
        print('Data was added to db successfully.')

    def flush(self):
        self.cursor.close()
        self.conn.close()
        self.cursor = None
        self.conn = None

In [31]:
famaga_gh_repo = FamagaKBRepo('Famaga Knowledge Map', r'C:\Users\MGroup\Downloads\langchain-400510-ac38916fe0c8.json')

In [32]:
discount_branch = famaga_gh_repo.get_discount_branch()

In [33]:
memory = MemoryKB()

In [34]:
memory.create_discount_table(discount_branch)

Data was added to db successfully.


In [36]:
raw_sql = """
SELECT * FROM discount
WHERE is_discount_specified = 0 AND is_customer_bought_previously = 0
"""

memory.cursor.execute(raw_sql)
rows = memory.cursor.fetchall()

for row in rows:
    print(row[4])

ask_specify_discount


In [30]:
db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
    { "role": "user", "content": f"""
Please build sql query that select one or more rows by presented data. Please ignore action field at sql request condition. 
Database: sqlite

Metadata:
sub_intent = Ask discount
is_discount_specified = false
is_customer_bought_previously = false

Use only the following table and columns:
Table: discount. Columns: sub_intent(text), is_margin_sufficient (boolean, null), is_discount_specified (boolean), 
is_customer_bought_previously (boolean), action (text)

Please do not write any commentaries or explanations, output only sql query.

Put the result at ```json``` format, like:
```sql
SELECT * FROM table_name
WHERE field1 = 'value_2'
""".strip()
    }])

```sql
SELECT * FROM discount
WHERE sub_intent = 'Ask discount' AND is_discount_specified = 0 AND is_customer_bought_previously = 0
```

--------------------

Note saved without feedback. ID: 0a348968-8d21-4276-bb90-0ec586f9edb0
Input tokens: 144 Output tokens: 36 Total price: 0.01$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

"```sql\nSELECT * FROM discount\nWHERE sub_intent = 'Ask discount' AND is_discount_specified = 0 AND is_customer_bought_previously = 0\n```"

**Base metadata**

- customer info
- parts inquiry
- 

In [37]:
"""
Kind regards
Austen Brunton BEng (Hons), MIMarEST
Technical Fleet Manager
WYM Group Ltd
Incorporating Wilson Yacht Management and Wilson Ship Management
New Email Address: austen@wymgroup.com
Web: www.wymgroup.com
Phone: +44 (0) 1482 648322
Mob: +44 (0) 7760665835
""".strip()

'Kind regards\nAusten Brunton BEng (Hons), MIMarEST\nTechnical Fleet Manager\nWYM Group Ltd\nIncorporating Wilson Yacht Management and Wilson Ship Management\nNew Email Address: austen@wymgroup.com\nWeb: www.wymgroup.com\nPhone: +44 (0) 1482 648322\nMob: +44 (0) 7760665835'

## Metadata classifier

In [ ]:
class DiscountInquiry
{
    [Field(description="Indicates if the margin is sufficient for discount")]
    public int is_margin_sufficient { get; set; }
}

In [ ]:
from typing import List, Dict, Optional
from dataclasses import dataclass, field

@dataclass
class DiscountInquiryMetadata:
    is_margin_sufficient: Optional[bool] = None
    is_discount_specified: Optional[bool] = None
    desired_discount: Optional[int] = None
    is_customer_bought_previously: Optional[bool] = None

@dataclass
class PartsInquiryMetadata:
    part_number: str
    brand: str

@dataclass
class CustomerInfo:
    email: str
    domain: str
    office_country: str
    phone: str

@dataclass
class MessageMetadata:
    intents: List[Dict[str, str]] = field(default_factory=list)
    parts_inquiries: List[PartsInquiryMetadata] = field(default_factory=list)
    customer_info: Optional[CustomerInfo] = None
    discount_inquiry: Optional[DiscountInquiryMetadata] = None

@dataclass
class Message:
    from_who: str
    message: str
    metadata: MessageMetadata

class MessagingHistory:
    def __init__(self):
        self.history: List[Message] = []
        self.discount_inquiry_state: Optional[DiscountInquiryMetadata] = None

    def add_message(self, from_who: str, message: str, metadata: Dict):
        """Add a message to the history, updating global inquiries state based on metadata."""
        # Convert metadata dict to MessageMetadata object
        message_metadata = self._parse_metadata(metadata)
        # Update global state if necessary
        if message_metadata.discount_inquiry:
            self._update_discount_inquiry_state(message_metadata.discount_inquiry)
            message_metadata.discount_inquiry = self.discount_inquiry_state
        new_message = Message(from_who, message, message_metadata)
        self.history.append(new_message)

    def _update_discount_inquiry_state(self, new_discount_metadata: DiscountInquiryMetadata):
        """Update global discount inquiry state."""
        if self.discount_inquiry_state is None:
            self.discount_inquiry_state = new_discount_metadata
        else:
            # Update existing state with any non-None values from the new state
            for field_name in vars(new_discount_metadata):
                new_value = getattr(new_discount_metadata, field_name)
                if new_value is not None:
                    setattr(self.discount_inquiry_state, field_name, new_value)

    def _parse_metadata(self, metadata: Dict) -> MessageMetadata:
        """Parse metadata dict to MessageMetadata object."""

        tracked_metadata = [metadata for metadata in self.global_metadata if metadata.state == State.Tracking]

        
        
        return MessageMetadata()  # Placeholder implementation

    def get_history(self) -> List[Message]:
        """Return the messaging history."""
        return self.history

# This approach requires implementing the _parse_metadata method to correctly
# convert incoming metadata dictionaries into strongly-typed MessageMetadata objects.


### Annotate fields

In [48]:
from dataclasses import dataclass, field
from typing import Optional, Any

def annotated_field(description: str, default: Any = None, include_in_instructions: bool = True):
    """Custom field for dataclasses with a description and optional inclusion in instructions."""
    return field(default=default, metadata={'description': description, 'include_in_instructions': include_in_instructions})


In [122]:
# @dataclass
# class DiscountInquiryMetadata:
#     is_margin_sufficient: Optional[bool] = annotated_field("Indicates if the margin is sufficient for discount", None)
#     is_discount_specified: Optional[bool] = annotated_field("True if the discount amount is specified by the customer", None)
#     desired_discount: Optional[int] = annotated_field("The percentage of the desired discount", None)
#     is_customer_bought_previously: Optional[bool] = annotated_field("True if the customer has made purchases previously", None, include_in_instructions=False)

#     @classmethod
#     def metadata_key(cls):
#         return "discount_inquiry"

#     @classmethod
#     def instruction(cls):
#         return ""


from pydantic import BaseModel, Field
from typing import Optional


class DiscountInquiryMetadata(BaseModel):
    is_margin_sufficient: Optional[bool] = None
    is_discount_specified: Optional[bool] = None
    desired_discount: Optional[int] = None
    is_customer_bought_previously: Optional[bool] = None

    @classmethod
    def metadata_key(cls):
        return "discount_inquiry"

    @classmethod
    def schema(cls):
        return  """
{
    "is_margin_sufficient": bool | null // Indicates if the margin is sufficient for discount",
    "is_discount_specified": bool | null // True if the discount amount is specified by the customer",
    "desired_discount": float | null // The percentage of the desired discount"
}
"""


In [ ]:
from typing import get_type_hints


def generate_schema(cls):
    """Generate JSON schema based on the data class fields and metadata, excluding certain fields."""
    schema = {}
    type_hints = get_type_hints(cls)  # Correctly retrieve type hints
    for field_name, field_info in cls.__dataclass_fields__.items():
        # Only include the field if it's meant to be in the instructions (or schema)
        if field_info.metadata.get('include_in_instructions', True):
            description = field_info.metadata.get('description', 'No description provided.')
            # Use type hints to get the field type
            field_type = type_hints[field_name]
            field_type_str = str(field_type).replace("typing.", "").replace("Optional[", "").replace("]", "")
            schema[field_name] = f"{field_type_str} // {description}"
    return schema

def generate_instructions(cls):
    """Generate human-readable instructions for filling out the data class fields, excluding certain fields."""
    instructions = []
    for field_name, field_info in cls.__dataclass_fields__.items():
        if field_info.metadata.get('include_in_instructions', True):
            description = field_info.metadata.get('description', 'No description provided.')
            instructions.append(f"- **{field_name}**: {description}")
    return "\n".join(instructions)

# Example usage
schema = generate_schema(DiscountInquiryMetadata)
instructions = generate_instructions(DiscountInquiryMetadata)

print(schema)
print(instructions)


In [120]:
# 

List


In [106]:
import json

print(json.dumps(schema, indent=4))

{
    "is_margin_sufficient": "typing.Optional[bool] // Indicates if the margin is sufficient for discount",
    "is_discount_specified": "typing.Optional[bool] // True if the discount amount is specified by the customer",
    "desired_discount": "typing.Optional[int] // The percentage of the desired discount"
}


In [78]:
executor = AgentExecutor(GPT_DB_LOGGER_PATH)

In [79]:
message_intents = executor.get_message_intents('Dear Kristine! Could you give us discount?')

```json
    {
        "id": "1",
        "body": "Dear Kristine! Could you give us discount?",
        "sign": "",
        "from": "customer",
        "intents": [
            {
                "intent": "Pricing and Quotes",
                "sub_intent": "Discount Inquiry"
            }
        ],
        "metadata_keys": [ "discount_inquiry"]
    
    }
```

--------------------

Note saved without feedback. ID: fd86829b-8ab9-4dd3-af49-c2837260de10
Input tokens: 344 Output tokens: 85 Total price: 0.02$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [81]:
message_intents.metadata_keys

['discount_inquiry']

In [124]:
def get_discount_metadata(msg, db_logger):
    return db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
        { "role": "user", "content": f"""
Please define the following values from the message

Message:
{msg}

Put the result at ```json``` format, like:
```json
{DiscountInquiryMetadata.schema()}
```
"""
        }])

In [125]:
discount_metadata = get_discount_metadata(
    'Dear Kristine! Could you give us discount?',
    executor.db_logger
)

```json
{
    "is_margin_sufficient": null,
    "is_discount_specified": false,
    "desired_discount": null
}
```

--------------------

Note saved without feedback. ID: 3af1d4e9-6c1b-439f-8e92-211e9ade9f92
Input tokens: 98 Output tokens: 31 Total price: 0.0$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [89]:
response_raw_json = select_json_block(discount_metadata)
DiscountInquiryMetadata(**response_raw_json)


DiscountInquiryMetadata(is_margin_sufficient=None, is_discount_specified=False, desired_discount=None, is_customer_bought_previously=None)

In [54]:
db_logger.create_completion(temperature=0.1, tags='intent_classifier', model='gpt-4', messages=[
    { "role": "user", "content": f"""
Please attach to each message in chat history tags, annotations, indexes, intents to better classification and search.
The message could have one or more intents.

Message:
```
Dear Kristine! Could you give us discount?
```

You can choose one of the following intents or figure out your own.
   - Order Processing -> Order Placement Confirmation
   - Order Processing -> Order Acceptance Confirmation
   - Pricing and Quotes -> Discount Inquiry
   - Pricing and Quotes -> Transport Cost Inquiry
   - Delivery and Shipping -> Delivery Method Discussion
   - Delivery and Shipping -> Discussing Logistics
   - Product Inquiry -> Availability Check

Also please define which metadata entites should be find from message.
The available keys:
- discount_inquiry: the metadata of discount discussions, could be specified at messages with Pricing and Quotes -> Discount Inquiry intent
- client_info: info about client email, phone, office, country, address
- part_inquiry: the info about parts and components (part number, brand, model, qty)


Put the result at ```json``` format, like:
```json
[
    {{
        "id": "<message_id>",
        "body": "<message body>", //only body without sign
        "sign": "<signature>", //message signature
        "from": "<message from>", //this should be one of two values: customer or manager
        "intents": [
            {{
                "intent": "<main intent>",
                "sub_intent": "<sub intent>",
                "keywords": ["word1", "word2"] //key words at base form (lemma)
            }}
        ],
        "metadata_keys": [ 'key1', 'key2']
    
    }}
]
```
Please put new lines symbols if it suitable for context \\n at body and sign fields.
""".strip()
    }])

```json
[
    {
        "id": "1",
        "body": "Dear Kristine! Could you give us discount?",
        "sign": "",
        "from": "customer",
        "intents": [
            {
                "intent": "Pricing and Quotes",
                "sub_intent": "Discount Inquiry",
                "keywords": ["discount"]
            }
        ],
        "metadata_keys": [ "discount_inquiry"]
    
    }
]
```

--------------------

Note saved without feedback. ID: fe786834-7b45-471c-97f9-ffca630353be
Input tokens: 365 Output tokens: 94 Total price: 0.02$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

'```json\n[\n    {\n        "id": "1",\n        "body": "Dear Kristine! Could you give us discount?",\n        "sign": "",\n        "from": "customer",\n        "intents": [\n            {\n                "intent": "Pricing and Quotes",\n                "sub_intent": "Discount Inquiry",\n                "keywords": ["discount"]\n            }\n        ],\n        "metadata_keys": [ "discount_inquiry"]\n    \n    }\n]\n```'

## Test

In [194]:
msg = """
Dear Sir or Madam,
Please advise if stators and rotors are available for the attached. The pump appears to be very similar to a Mono pump
Kind regards
Austen Brunton BEng (Hons), MIMarEST
Technical Fleet Manager
WYM Group Ltd
Incorporating Wilson Yacht Management and Wilson Ship Management
New Email Address: austen@wymgroup.com
Web: www.wymgroup.com
Phone: +44 (0) 1482 648322
Mob: +44 (0) 7760665835
"""

msg1 = """
Good Afternoon Kristine,
 
Please see below proof of payment that should appear in your account shortly.

 
Have a great weekend!
 
Many thanks,
Miki
"""

msg2 = """
Dear Kristine Gergaia,


Thank you for your offer. 

Please send me offer for the coil 0000000.840423050 (Norgren)  5pcs.

Delivery time 5-6 weeks. It`s correct?

 
Best regards, 
Yuri Avdonjushkin

OÜ Anveide reg,10078552

VAT:  EE100229833

Kärberi 16-53, 13812 Tallinn, Estonia

Tel. +372 5153828

E-mail:  juri.avdonyushkin@gmail.com  
"""

msg3 = """
Afternoon

 

Many thanks for your email – please can you confirm if the price quoted includes the price for the certificates?

 

Tim Carde

RSC Spares Ltd

Unit 91 Portmanmoor Rd Ind Est

Cardiff

CF24 5HB

Tel: 029 20489555

Email: tim@rsc-spares.co.uk
"""

In [195]:
parsed_messages = get_message_intents(msg3)

```json
[
    {
        "id": "1",
        "body": "Afternoon\n\nMany thanks for your email – please can you confirm if the price quoted includes the price for the certificates?",
        "sign": "Tim Carde\nRSC Spares Ltd\nUnit 91 Portmanmoor Rd Ind Est\nCardiff\nCF24 5HB\nTel: 029 20489555\nEmail: tim@rsc-spares.co.uk",
        "from": "customer",
        "intents": [
            {
                "intent": "Pricing and Quotes",
                "sub_intent": "Price Confirmation",
                "branch": "Confirmation of Price Including Certificates"
            }
        ]
    
    }
]
```

--------------------

Note saved without feedback. ID: 4e4fabdf-2bbc-41e2-89cb-3ab572b27613
Input tokens: 460 Output tokens: 153 Total price: 0.02$




Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [192]:
parsed_messages = get_message_intents(msg2)

```json
[
    {
        "id": "1",
        "body": "Dear Kristine Gergaia,\n\nThank you for your offer. \n\nPlease send me offer for the coil 0000000.840423050 (Norgren)  5pcs.\n\nDelivery time 5-6 weeks. It`s correct?\n\nBest regards,", 
        "sign": "Yuri Avdonjushkin\n\nOÜ Anveide reg,10078552\n\nVAT:  EE100229833\n\nKärberi 16-53, 13812 Tallinn, Estonia\n\nTel. +372 5153828\n\nE-mail:  juri.avdonyushkin@gmail.com", 
        "from": "customer", 
        "intents": [
            {
                "intent": "Order Processing",
                "sub_intent": "Order Placement Confirmation",
                "branch": "Confirmation of Specific Offer"
            },
            {
                "intent": "Delivery and Shipping",
                "sub_intent": "Delivery Time Inquiry",
                "branch": "Inquiry About Delivery Time"
            }
        ]
    
    }
]
```

--------------------

Note saved without feedback. ID: 0b48087a-069b-4d9f-96e9-bd4910dbb8de
Input tokens: 509

Textarea(value='', description='Feedback:', layout=Layout(height='80px', width='70%'), placeholder='Type your …

Button(button_style='success', description='Submit Feedback', style=ButtonStyle(), tooltip='Click to submit fe…

In [ ]:
msg_details = get_details_from_message(msg)

In [ ]:
intents_str = '\n'.join([f'{intent.intent} -> {intent.sub_intent}' for intent in parsed_messages[0].intents])
raw_sql = build_sql_query(msg_details, intents_str)

In [133]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

ghconv = GoogleSheetToSQLiteConverter(
    spreadsheet='Famaga Knowledge Map', 
    credentials_path=r'C:\Users\MGroup\Downloads\langchain-400510-ac38916fe0c8.json')

sheet_data = ghconv.get_sheet_data('Classify parts')

product_inquiries = [
    ProductInquiry(
        intent=item['Intent'],
        sub_intent=item['Sub-intent'],
        is_brand_name=item['Brand name'],
        is_part_number=item['Part Number'],
        action=item['Action']
    )
    for item in sheet_data
]

init_sqlite_db(conn, product_inquiries)


Data was added to db successfully.


In [126]:
cursor.execute(raw_sql)

In [134]:
cursor.execute(raw_sql)
rows = cursor.fetchall()

for row in rows:
    print(row)

('Product Inquiry', 'Availability Check', 1, 0, 'ask_specify_part_number')


In [ ]:
MsgHtmlExtracter.extract_messages_from_raw_html()

In [159]:
messages_html = deals_rep.get_messaging_html(415947)
# resp = client.handle_messages(123,messages_html)

In [160]:
messages = MsgHtmlExtracter.extract_messages_from_raw_html(messages_html)

In [ ]:
print('\n\n'.join([f'Message:{m}' if 'for orders under 150 Euro' not in m  else 'Message: Offer' for m in messages[::-1]]))

In [167]:
client = AgentsAPIClient('https://neon-dev.us')


In [ ]:
client.handle_html_messages(deal_id="322", messages_html=messages_html)

In [170]:
client.handle_list_messages(deal_id="322", messages=[
"""
Dear Sir or Madam,
Please advise if stators and rotors are available for the attached. The pump appears to be very similar to a Mono pump
Kind regards
Austen Brunton BEng (Hons), MIMarEST
Technical Fleet Manager
WYM Group Ltd
Incorporating Wilson Yacht Management and Wilson Ship Management
New Email Address: austen@wymgroup.com
Web: www.wymgroup.com
Phone: +44 (0) 1482 648322
Mob: +44 (0) 7760665835
"""
])

{'output': {'parts': [{'amount': None,
    'brand_name': 'Mono',
    'part_number': None}],
  'client': {'country': 'United Kingdom',
   'domain': 'wymgroup.com',
   'email': 'austen@wymgroup.com',
   'office_country': 'United Kingdom'},
  'deal_id': None,
  'message_id': None,
  'agent_task_id': None},
 'action': 'classify_parts',
 'logs': [{'step_name': 'classify_request',
   'log': 'Decision Point: Classify customer request\nCondition: Did the customer leave a request indicating specific parts?\nObservation: Yes, the customer specified stators and rotors for a pump similar to a Mono pump.\n\nAction: Classify parts from request',
   'task_id': 1651},
  {'step_name': 'classify_parts',
   'log': '```json\n{\n    "parts": [\n        {\n           "amount": null,\n           "brand_name": "Mono",\n           "part_number": null\n        }\n    ],\n    "client": {\n        "country": "United Kingdom",\n        "domain": "wymgroup.com",\n        "email": "austen@wymgroup.com",\n        "of

## Work with Scenarios

In [219]:
from typing import List, Optional, Union
from pydantic import BaseModel

class Position(BaseModel):
    position: str
    description: str
    article: str
    qty: int
    discount_percents: str
    discounted_price: float
    sum: float
    delivery_time_from_days: int
    delivery_time_to_days: int

class Total(BaseModel):
    payment: float

class Offer(BaseModel):
    positions: List[Position]
    total: Total

# Since messages can have either a text message or an offer, we use Union for typing
class Message(BaseModel):
    from_: str = None  # Use 'from_' because 'from' is a reserved keyword in Python
    message: Optional[str] = None
    offer: Optional[Offer] = None

    # Customizing the field names to match the JSON structure
    class Config:
        fields = {'from_': 'from'}

class Scenario(BaseModel):
    messaging_history: List[Message]

class Scenarios(BaseModel):
    scenarios: List[Scenario]


C:\Users\MGroup\components_agent_sales\venv\Lib\site-packages\pydantic\_internal\_config.py:321: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [229]:
client_deals_rep.optimized_get_deals_with_exceeds_price_with_gap(min_day_gap=1)

,client_id,articul,id
49789,80323,0000000150011000,456020
49514,105309,00004669,455147
54036,92227,0014600000000000,462640
33873,103498,0022111,430545
44625,113520,007.000.000.002.204,445624
...,...,...,...
21772,97560,ZK311011,414034
21773,97560,ZK311012,414034
21753,97560,ZK451006,414034
21754,97560,ZK451007,414034


In [239]:
client_history = client_deals_rep.get_client_history(67197)

In [ ]:
client_history

In [246]:
from collections import defaultdict
from typing import Dict, List, Set


def find_articles_bought_multiple_times(data: Dict) -> Dict[str, Set]:
    article_dates = defaultdict(set)

    for purchase_date, transactions in data.items():
        for transaction in transactions:
            article_id = transaction['articul']
            # Record the date for each article_id, automatically avoiding duplicates due to the set
            article_dates[article_id].add(purchase_date)

    # Filter out articles bought only once
    return {article_id: dates for article_id, dates in article_dates.items() if len(dates) > 1}


def print_duplicated_items_grouped_by_date(data):
    # Step 1: Group transactions by article ID
    articles = defaultdict(list)
    for date, transactions in data.items():
        for transaction in transactions:
            article_id = transaction['articul']
            articles[article_id].append((date, transaction))
    
    # Step 2: Identify duplicates and group them by date
    duplicates_by_date = defaultdict(lambda: defaultdict(list))
    for article_id, transactions in articles.items():
        if len(transactions) > 1:  # Only consider items bought more than once
            for date, transaction in transactions:
                duplicates_by_date[date][article_id].append(transaction)

    # Step 3: Print or return the grouped duplicates
    print("**Client purchase history:**")
    for date, articles in sorted(duplicates_by_date.items()):
        print(f"Date: {date}")
        for article_id, transactions in articles.items():
            for transaction in transactions:
                brand_title = transaction['brand_title'] if transaction['brand_title'] else "None"
                margin = transaction['margin']
                sell_price = transaction['price_sell']
                qty = transaction['amount']
                print(f"{transaction['id']} ({article_id} {brand_title}) margin: {margin:.2f}%, sell: {sell_price}$ qty. {qty}")
        print()


def get_duplicated_items_grouped_by_date(data):
    # Step 1: Group transactions by article ID
    articles = defaultdict(list)
    for date, transactions in data.items():
        for transaction in transactions:
            article_id = transaction['articul']
            articles[article_id].append((date, transaction))
    
    # Step 2: Identify duplicates and prepare them for re-structuring
    duplicates_by_date = defaultdict(list)
    for article_id, transactions in articles.items():
        if len(transactions) > 1:  # Only consider items bought more than once
            for date, transaction in transactions:
                duplicates_by_date[date].append(transaction)

    # Return the structured data similar to the input format
    return duplicates_by_date


articles_bought_multiple_times = find_articles_bought_multiple_times(client_history)
for article_id, dates in articles_bought_multiple_times.items():
    print(f"Article ID: {article_id} was bought on {len(dates)} different dates.")


Article ID: ZP516A02RD01 was bought on 2 different dates.
Article ID:  was bought on 4 different dates.
Article ID: 6ES7972-0BA52-0XA0 was bought on 2 different dates.
Article ID: 6ES7155-6BA01-0CN0 was bought on 2 different dates.
Article ID: 6ES7193-6BP00-0BA0 was bought on 2 different dates.
Article ID: 6EP7133-6AB00-0BN0 was bought on 2 different dates.


In [254]:
import json


duplicated_data = get_duplicated_items_grouped_by_date(client_history)

In [ ]:
import math
from pandas import isna


def flatten_data(data):
    flat_list = []
    for date, transactions in data.items():
        for transaction in transactions:
            # Create a copy to avoid modifying the original data
            flat_transaction = transaction.copy()
            # Add the purchase date as a field in each transaction
            flat_transaction['purchase_date'] = date.isoformat()
            for key, value in flat_transaction.items():
                if isna(value):  # Checks for NaN and None, replaces with Python's None if true
                    flat_transaction[key] = None
            flat_list.append(flat_transaction)
    return flat_list


flattened_data = flatten_data(client_history)
flattened_data

In [ ]:
print(json.dumps(flattened_data, cls=CustomEncoder, indent=4))

In [ ]:
import json
from datetime import date


duplicated_items_grouped_by_date = dict(duplicated_data)


def serialize_data(data):
    # Convert dates to string in keys
    serialized_data = {date.isoformat(): transactions for date, transactions in data.items()}
    return serialized_data

class CustomEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, date):
            return obj.isoformat()
        return json.JSONEncoder.default(self, obj)

# duplicated_items_json = json.dumps(duplicated_items_grouped_by_date, cls=CustomEncoder, indent=4)

# print(duplicated_items_json)

serialized_data = serialize_data(duplicated_items_grouped_by_date)
duplicated_items_json = json.dumps(serialized_data, cls=CustomEncoder, indent=4)

print(duplicated_items_json)

In [250]:
print(ClientStatisticsService.get_purchase_history_str(get_duplicated_items_grouped_by_date(client_history)))

**Client purchase history:**
Date: 2023-01-27
380825 (ZP516A02RD01 DELIMON / BIJUR DELIMON) margin: 23.22%, sell: 3887.0$ qty. 1
380825 ( None) margin: 15.29%, sell: 850.0$ qty. 1

Date: 2023-02-23
387541 (ZP516A02RD01 DELIMON / BIJUR DELIMON) margin: 24.37%, sell: 5262.5$ qty. 1

Date: 2023-07-28
426634 ( None) margin: 41.18%, sell: 170.0$ qty. 1
426634 (6ES7972-0BA52-0XA0 Siemens) margin: 20.96%, sell: 38.7$ qty. 1
426634 (6ES7155-6BA01-0CN0 Siemens) margin: 18.72%, sell: 287.12$ qty. 1
426634 (6ES7193-6BP00-0BA0 Siemens) margin: 18.71%, sell: 14.22$ qty. 2
426634 (6EP7133-6AB00-0BN0 Siemens) margin: 7.32%, sell: 121.53$ qty. 1

Date: 2023-09-14
438515 ( None) margin: 28.57%, sell: 140.0$ qty. 1
438515 (6ES7972-0BA52-0XA0 Siemens) margin: 31.9%, sell: 42.89$ qty. 1
438515 (6ES7155-6BA01-0CN0 Siemens) margin: 30.09%, sell: 310.0$ qty. 1
438515 (6ES7193-6BP00-0BA0 Siemens) margin: 31.92%, sell: 15.76$ qty. 3
438515 (6EP7133-6AB00-0BN0 Siemens) margin: 29.44%, sell: 130.0$ qty. 1

Date:

In [ ]:
get_duplicated_items_grouped_by_date(client_history)

In [241]:
print(ClientStatisticsService.get_purchase_history_str(client_history))

**Client purchase history:**
Date: 2023-01-27
380825 (ZP516A02RD01 DELIMON / BIJUR DELIMON) margin: 23.22%, sell: 3887.0$ qty. 1
380825 ( None) margin: 15.29%, sell: 850.0$ qty. 1

Date: 2023-02-23
387541 (ZP516A02RD01 DELIMON / BIJUR DELIMON) margin: 24.37%, sell: 5262.5$ qty. 1

Date: 2023-05-02
403142 (6SL3220-2YE56-0CF0 Siemens) margin: 20.8%, sell: 15081.53$ qty. 4
403142 (3VA2580-6HL32-0AA0 Siemens) margin: 20.8%, sell: 1398.51$ qty. 4
403142 (3KD4830-0QE10-0 Siemens) margin: 20.8%, sell: 541.65$ qty. 4
403142 (8UD1151-3AF21 Siemens) margin: 20.81%, sell: 42.2$ qty. 4
403142 (6SL3000-0CE36-3AA0 Siemens) margin: 20.8%, sell: 1109.16$ qty. 4

Date: 2023-07-28
426634 ( None) margin: 41.18%, sell: 170.0$ qty. 1
426634 (6EP1334-1LB00 Siemens) margin: 27.59%, sell: 88.87$ qty. 1
426634 (6ES7222-1HF32-0XB0 Siemens) margin: 35.15%, sell: 128.3$ qty. 1
426634 (6ES7972-0BA52-0XA0 Siemens) margin: 20.96%, sell: 38.7$ qty. 1
426634 (6ES7155-6BA01-0CN0 Siemens) margin: 18.72%, sell: 287.12$ q